In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import numpy as np
from scipy.interpolate import interp1d

C:\Users\thewe\anaconda3\envs\crocusenv\Lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()
Cannot find header.dxf (GDAL_DATA is not defined)


In [ ]:
file_path = '2024-07-22_1427.raw_flight_history.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Strip any leading/trailing whitespace from column names
data.columns = data.columns.str.strip()

# Convert Latitude, Longitude, and Wind Speed to numeric, forcing errors to NaN
data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce')
data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce')
data['Speed (m/s)'] = pd.to_numeric(data['Speed (m/s)'], errors='coerce')

# Interpolate wind speed for rows with missing lat/lon
missing_lat_lon = data[(data['Latitude'].isna()) | (data['Longitude'].isna())]
non_missing_lat_lon = data.dropna(subset=['Latitude', 'Longitude'])

In [ ]:
if not missing_lat_lon.empty and not non_missing_lat_lon.empty:
    # Interpolation function for wind speed
    speed_interp_func = interp1d(non_missing_lat_lon.index, non_missing_lat_lon['Speed (m/s)'], kind='linear', fill_value='extrapolate')
    
    # Apply the interpolation function to the indices of missing lat/lon rows
    data.loc[missing_lat_lon.index, 'Speed (m/s)'] = speed_interp_func(missing_lat_lon.index)

# Clean the lat and lon data by removing rows with NaN values
cleaned_data = data.dropna(subset=['Latitude', 'Longitude', 'Speed (m/s)'])

# Verify the cleaned data
print(cleaned_data[['Latitude', 'Longitude', 'Speed (m/s)']].head())

# Define the extent of the map (lon_min, lon_max, lat_min, lat_max)
extent = [-87.65, -87.60, 41.83, 41.875]

# Create a Cartopy map with Google tiles
tiler = cimgt.GoogleTiles(style='satellite')
fig = plt.figure(figsize=(10, 10), dpi=300)
ax = plt.axes(projection=tiler.crs)
ax.set_extent(extent)

# Add the Google satellite tiles to the map
ax.add_image(tiler, 15)

# Create a scatter plot with color coding for wind speed
sc = ax.scatter(cleaned_data['Longitude'], cleaned_data['Latitude'], c=cleaned_data['Speed (m/s)'], cmap='jet', s=10, transform=ccrs.PlateCarree())

# Add color bar
cbar = plt.colorbar(sc, ax=ax, orientation='vertical', shrink=0.7, pad=0.02)
cbar.set_label('Wind Speed (m/s)')

# Add start and end text
start_point = cleaned_data.iloc[0]
end_point = cleaned_data.iloc[-1]

ax.text(start_point['Longitude'], start_point['Latitude'], 'Start', transform=ccrs.PlateCarree(), fontsize=10, ha='right')
ax.text(end_point['Longitude'], end_point['Latitude'], 'End', transform=ccrs.PlateCarree(), fontsize=10, ha='left')

# Add title and labels
plt.title('Trajectory of the Sounding with Wind Speed, 1427z, July 22nd')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('sounding125422.png', bbox_inches='tight', dpi=300)
# Display the plot
plt.show()
